# ARES, Penn Library Course Reserves Issue
> Summary of Issue: the ARES LTI which embeds library references in canvas is not visible for the 'Observer' role. However in batch updates, the 'Observer' role should be mapped to student.


#### Example of Issue

Course:  https://upenn.test.instructure.com/courses/1418936  
What the view should be:

<img src='../images/ares_issue.png' width="650" height="650" >


When one masquerades as a user with the 'Observer' Role in the Canvas side navigation UI 
the LTI is not permitting access so it is not showing up in the sidebar. However, one can still access the tool with the URL:

<img src="../images/ares_observer.png" width="650" height="650">




### Investigation into permissions
It appears that there is no authentication issue for 'Observers' with ARES since they can access with the URL. The question then becomes why aren't 'Observer's given permission to see the navigation tab to the tool in Canvas. The UI of the navigation setting is rather limited and only defines visibility in terms of student and instructor roles. 


#### Visibility Defined in the Canvas API Documentation
> **visibility**: 'public', 'members', 'admins':  This specifies what types of users will see the link in the course navigation. "public" means anyone accessing the course, "members" means only users enrolled in the course, and "admins" means only Teachers, TAs, Designers and account admins will see the link. ([source](https://canvas.instructure.com/doc/api/file.navigation_tools.html))


In [2]:
# just some important imports ,, run this cell when u start 
import json
import re
import requests
from pprintpp import pprint as pp
import ipywidgets as widgets
from IPython.core.display import display, HTML
import time

with open('../live_envir.json') as json_f:
    data = json.load(json_f)
    key = data['api_key']
    print("...loaded my key and did some package imports...")
    
domain = 'https://upenn.test.instructure.com'
headers = {
    'Authorization': 'Bearer %s' % (key) ,
}

...loaded my key and did some package imports...


#### Let's print out a the metadata about the tab for the 'label': 'Penn Library Course Reserves' ( ARES )

In [3]:
#https://canvas.instructure.com/doc/api/tabs.html
course_number = '1418936'
external_tool_id = '62901'
url = '%s/api/v1/courses/%s/tabs' % (domain,course_number)

response = requests.get(url, headers=headers).json()
result = [ x for x in response if x['label'] == 'Penn Library Course Reserves']
pp(result)

[
    {
        'full_url': 'https://upenn.test.instructure.com/courses/1418936/external_tools/62901',
        'html_url': '/courses/1418936/external_tools/62901',
        'id': 'context_external_tool_62901',
        'label': 'Penn Library Course Reserves',
        'position': 8,
        'type': 'external',
        'url': 'https://upenn.test.instructure.com/api/v1/courses/1418936/external_tools/sessionless_launch?id=62901&launch_type=course_navigation',
        'visibility': 'public',
    },
]



#### Unpacking the result

'visibility':'admins' means that the tool is only visibile to admins -- disabled currently  
'visibility':'public' means that the tool has been activated and is visible to everyone who can access the course.   

For the 'Penn Library Course Reserves' navigation tab, the metadata is:  
(note if the rest of the notebook has been run already this may not match the above)  

```
{
    'full_url': 'https://upenn.test.instructure.com/courses/1418936/external_tools/62901',
    'html_url': '/courses/1418936/external_tools/62901',
    'id': 'context_external_tool_62901',
    'label': ' Penn Library Course Reserves',
    'position': 8,
    'type': 'external',
    'url': 'https://upenn.test.instructure.com/api/v1/courses/1418936/external_tools/sessionless_launch?id=62901&launch_type=course_navigation',
    'visibility': 'members',
}```  


**Note:** The 'visibility' is set to 'members' whereas the other tabs are set to 'public'. This setting is probably due to the original set-up of the tool which apparently did not follow standard protocol.  

Testing with the API has shown that the 'visibility' of the tab corresponding to the tool is not permitted to be changed at the course level.   
See example bellow where attempts to change the visibility to 'admin' do not go through:

In [4]:
# Let's try to update a tab for a course ... 
#this probably wont work because 'visibility' isnt a documented parameter

tab_id = 'context_external_tool_62901'
course_number = '1418936'
external_tool_id = '62901'
url = '%s/api/v1/courses/%s/tabs/%s' % (domain,course_number,tab_id)

files= {
    'visibility':(None,'admin')
}

response = requests.put(url, headers=headers,files=files).json()
pp(response)

# blerg this doesn't work ..

{
    'full_url': 'https://upenn.test.instructure.com/courses/1418936/external_tools/62901',
    'html_url': '/courses/1418936/external_tools/62901',
    'id': 'context_external_tool_62901',
    'label': 'Penn Library Course Reserves',
    'position': 8,
    'type': 'external',
    'url': 'https://upenn.test.instructure.com/api/v1/courses/1418936/external_tools/sessionless_launch?id=62901&launch_type=course_navigation',
    'visibility': 'public',
}


### Stepping away from the navigation permissions, let's see what the info associated with the tool for this course

In [6]:
# Let's look at the info associated with the tool for this course
course_number = '1418936'
external_tool_id = '62901'
url = '%s/api/v1/courses/%s/external_tools' % (domain,course_number)
files = {
    'include_parents': (None,'true'),
}
response = requests.get(url, headers=headers,files=files).json()
tool = [ x for x in response if x['id']==62901 ]
tool[0]['consumer_key'] = 'confidential'
pp(tool)

[
    {
        'account_navigation': None,
        'assignment_edit': None,
        'assignment_menu': None,
        'assignment_selection': None,
        'assignment_view': None,
        'collaboration': None,
        'consumer_key': 'confidential',
        'course_assignments_menu': None,
        'course_home_sub_navigation': None,
        'course_navigation': {
            'default': 'disabled',
            'label': ' Penn Library Course Reserves',
            'selection_height': 400,
            'selection_width': 800,
            'text': ' Penn Library Course Reserves',
            'visibility': 'members',
        },
        'course_settings_sub_navigation': None,
        'created_at': '2014-08-05T14:26:40Z',
        'custom_fields': {},
        'description': 'View reserved content for this course',
        'discussion_topic_menu': None,
        'domain': None,
        'editor_button': None,
        'file_menu': None,
        'global_navigation': None,
        'homework_submissi

### Let's try to edit the 'visibility' of the tool at the course level. 

In [7]:
# Let's try to edit the 'visibility' here at the course_id
course_number = '1418936'
external_tool_id = '62901'
url = '%s/api/v1/courses/%s/external_tools/%s' % (domain,course_number,external_tool_id)
files = {
    'user_navigation[visibility]': (None,'public')
}
response = requests.get(url, headers=headers).json()
pp(response)
# we can't even see the meta data for this LTI!
#   this error occurs because the LTI is not installed at the course level...? 

{
    'error_report_id': 411150948,
    'errors': [{'message': 'The specified resource does not exist.'}],
}


#### Since the tool does not exist individually at the course level but rather, at the account level, the tool's permissions should be changed there. 

In [8]:
# Let's try to edit the 'visibility' at the account level
external_tool_id = '62901'
url = '%s/api/v1/accounts/self/external_tools/%s' % (domain,external_tool_id)
files = {
    'course_navigation[visibility]': (None,'public')
}
response = requests.put(url, headers=headers,files=files).json()
response['consumer_key'] = 'confidential'
pp(response)

#this has fixed the visibility issue for 'Observers' 
#but it should be noted that instead of having the course_navigation[visibility] change ... it deleted ...

{
    'account_navigation': None,
    'assignment_edit': None,
    'assignment_menu': None,
    'assignment_selection': None,
    'assignment_view': None,
    'collaboration': None,
    'consumer_key': 'confidential',
    'course_assignments_menu': None,
    'course_home_sub_navigation': None,
    'course_navigation': {
        'label': 'Penn Library Course Reserves',
        'selection_height': 400,
        'selection_width': 800,
    },
    'course_settings_sub_navigation': None,
    'created_at': '2014-08-05T14:26:40Z',
    'custom_fields': {},
    'description': 'View reserved content for this course',
    'discussion_topic_menu': None,
    'domain': None,
    'editor_button': None,
    'file_menu': None,
    'global_navigation': None,
    'homework_submission': None,
    'id': 62901,
    'link_selection': None,
    'migration_selection': None,
    'module_menu': None,
    'name': 'Penn Library Course Reserves',
    'not_selectable': False,
    'post_grades': None,
    'privacy_lev

## In closing
This has fixed the visibility issue for 'Observers' for each course that uses ARES. 


It should be noted that instead of having the course_navigation \[visibility] change ... the metadata for that attribute was deleted all together. I am unclear why but it may be due to the the initial configuration of the LTI which was rather "hacky".  

Next Steps:
1. Find all courses that use ARES.
    - get a list of all courses this semester
     - iterate through and see if it uses the tool and if so add the course no. to the list 
2. Find all students that are observers in the courses from step 1 and masquerade as them and pull the html and validate if the Reserves tab is present
3. validate for the non-observers that their view is still ok?


In [8]:

current_term_id = '5911'
external_tool_id = '62901'
url = '%s/api/v1/accounts/self/courses' % (domain)
data = {
    'enrollment_type[]': 'observer',
    'enrollment_state': 'active',
    'completed': False,
    'include[]': 'observed_users',
    'enrollment_term_id': current_term_id
}
response = requests.get(url, headers=headers,data=data)
r=response.json()

parsed_links = requests.utils.parse_header_links(response.headers['Link'].rstrip('>').replace('>,<', ',<'))
pp(parsed_links)
display(HTML('<p><b>queried:</b> %s</p><p><b>result(%s):</b></p>' % (url,len(r))))
pp(r)




[
    {
        'rel': 'current',
        'url': 'https://upenn.test.instructure.com/api/v1/accounts/self/courses?page=1&per_page=10',
    },
    {
        'rel': 'next',
        'url': 'https://upenn.test.instructure.com/api/v1/accounts/self/courses?page=2&per_page=10',
    },
    {
        'rel': 'first',
        'url': 'https://upenn.test.instructure.com/api/v1/accounts/self/courses?page=1&per_page=10',
    },
    {
        'rel': 'last',
        'url': 'https://upenn.test.instructure.com/api/v1/accounts/self/courses?page=8&per_page=10',
    },
]


[
    {
        'account_id': 97010,
        'apply_assignment_group_weights': False,
        'calendar': {
            'ics': 'https://upenn.test.instructure.com/feeds/calendars/course_fcmqsDQ36EdrocYrfRaW5KreCRgTKQaglVI8d3wT.ics',
        },
        'course_code': 'Coaching PHL 43',
        'created_at': '2018-02-26T17:38:30Z',
        'default_view': 'syllabus',
        'end_at': '2019-02-01T04:59:00Z',
        'enrollment_term_id': 5911,
        'grading_standard_id': None,
        'hide_final_grades': False,
        'id': 1395932,
        'integration_id': None,
        'is_public': False,
        'is_public_to_auth_users': False,
        'name': 'Coaching PHL 43',
        'public_syllabus': False,
        'public_syllabus_to_auth': False,
        'restrict_enrollments_to_course_dates': True,
        'root_account_id': 96678,
        'sis_course_id': 'ECFP-43e',
        'sis_import_id': None,
        'start_at': '2018-05-02T04:00:00Z',
        'storage_quota_mb': 2000,
        'ti

## References
> https://canvas.instructure.com/doc/api/external_tools.html
> https://canvas.instructure.com/doc/api/file.masquerading.html
> https://canvas.instructure.com/doc/api/file.navigation_tools.html    
> API for accessing and configuring external tools on accounts and courses. "External tools" are IMS LTI links: http://www.imsglobal.org/developers/LTI/index.cfm  

In [13]:
external_tool_id = '62901'
url = '%s/api/v1/accounts/self/terms' % (domain)

response = requests.get(url, headers=headers).json()

pp(response)

{
    'enrollment_terms': [
        {
            'created_at': '2011-05-12T02:18:38Z',
            'end_at': None,
            'grading_period_group_id': None,
            'id': 410,
            'name': 'Default Term (PennGSE)',
            'sis_import_id': None,
            'sis_term_id': 'canvas_default-G',
            'start_at': None,
            'workflow_state': 'active',
        },
        {
            'created_at': '2012-05-21T13:46:11Z',
            'end_at': None,
            'grading_period_group_id': None,
            'id': 2244,
            'name': 'Penn Term',
            'sis_import_id': None,
            'sis_term_id': 'penn-term',
            'start_at': None,
            'workflow_state': 'active',
        },
        {
            'created_at': '2013-02-22T21:02:24Z',
            'end_at': None,
            'grading_period_group_id': None,
            'id': 4373,
            'name': 'Ongoing (Communities)',
            'sis_import_id': None,
            'sis_term_id